In [ ]:
"""
This notebook evaluates the model's perfomance on the test set using the following metrics: exact match, BLEU, CODEBLEU.
"""

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.5 MB/s eta 0:00:00


In [ ]:
! pip install transformers
!pip install tree_sitter==0.2.0
! git clone -q https://github.com/microsoft/CodeXGLUE.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.4/110.4 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tree_sitter: filename=tree_sitter-0.2.0-cp311-cp311-linux_x86_64.whl size=389535 sha256=fb9a9828e14d0e9b3f112a07271e6b4c250aba537dea2c02668ab71862322c53
  Stored in directory: /root/.cache/pip/wheels/d9/6e/e2/b0126ad4f531cf09749b69518118f0ebf7bf3134ed91c71abb
Successfully built tree_sitter


In [ ]:
import sys
def print_progress(percent):
    width = 50  # Width of the progress bar
    filled = int(width * percent / 100)
    bar = "#" * filled + "-" * (width - filled)
    sys.stdout.write(f"\r[{bar}] {percent}%")
    sys.stdout.flush()

In [ ]:
import pandas as pd
# df_results = pd.read_csv("predictions_no_tab.csv")
df_results = pd.read_csv("predictions_with_tab.csv")

In [ ]:
df_results.head().drop(columns=["Unnamed: 0"])

,input_function,expected_if_statement,predicted_if_statement,predicted_full_method,target_full_method
0,"def read ( self , count = True , timeout = Non...",if ignore_timeouts and is_timeout ( e ) :,if ignore_timeouts and is_noerr ( e ) :,"def read ( self , count = True , timeout = Non...","def read ( self , count = True , timeout = Non..."
1,"def _cache_mem ( curr_out , prev_mem , mem_len...",if prev_mem is None :,if mem_len < len ( curr_out ) :,"def _cache_mem ( curr_out , prev_mem , mem_len...","def _cache_mem ( curr_out , prev_mem , mem_len..."
2,def filtered ( gen ) : <TAB> for example in ge...,if example_len > max_length :,if example_len > max_length :,def filtered ( gen ) : <TAB> for example in ge...,def filtered ( gen ) : <TAB> for example in ge...
3,"def search ( self , query ) : <TAB> # ""Search....","if item . get ( ""type"" , """" ) == ""audio"" :","if ""guide_id"" in item :","def search ( self , query ) : <TAB> # ""Search....","def search ( self , query ) : <TAB> # ""Search...."
4,"def _check_script ( self , script , directive ...","if var . must_contain ( ""/"" ) :",if var . name in self . _untrusted_vars :,"def _check_script ( self , script , directive ...","def _check_script ( self , script , directive ..."


In [ ]:
predicted_full_methods = list(df_results["predicted_full_method"])
target_full_methods = list(df_results["target_full_method"])

In [ ]:
assert len(predicted_full_methods) == len(target_full_methods)

In [ ]:
assert len(predicted_full_methods) == 5000

In [ ]:
# get CODEBleu scores for each prediction-target pair
code_bleu_list = []
for i in range(5000):
    with open("predictions.txt", "w", encoding="utf-8") as f:
      f.write(predicted_full_methods[i])
    with open("targets.txt", "w", encoding="utf-8") as f:
      f.write(target_full_methods[i])
    metrics = !cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && python calc_code_bleu.py --refs /content/targets.txt --hyp /content/predictions.txt --lang python --params 0.25,0.25,0.25,0.25
    code_bleu_score = float(metrics[-1][16:])
    code_bleu_list.append(code_bleu_score)
    #print(code_bleu_score)
    if i % 100 == 0:
      print_progress((i+1)/5000*100)
print_progress(100)

[##################################################] 100%

In [ ]:
df_results["code_bleu"] = code_bleu_list

In [ ]:
df_results["code_bleu"] = df_results["code_bleu"] * 100

In [ ]:
import sacrebleu

bleu_score_list = []
for i in range(5000):
  hypotheses = [predicted_full_methods[i]]
  references = [[target_full_methods[i]]]

  score = sacrebleu.corpus_bleu(hypotheses, references)
  bleu_score = str(score).split()[2]
  bleu_score_list.append(bleu_score)
  if i % 100 == 0:
    print_progress((i+1)/5000*100)
print_progress(100)

[##################################################] 100%

In [ ]:
df_results["bleu_4"] = bleu_score_list

In [ ]:
# get whether the target and the prediction is an exact match for each target-prediction pair
exact_match_list = []

for i in range(5000):
    is_exact_match = predicted_full_methods[i] == target_full_methods[i]
    exact_match_list.append(is_exact_match)

In [ ]:
df_results["exact_match"] = exact_match_list

In [ ]:
df_results

,Unnamed: 0,input_function,expected_if_statement,predicted_if_statement,predicted_full_method,target_full_method,code_bleu,bleu_4,exact_match
0,0,"def read ( self , count = True , timeout = Non...",if ignore_timeouts and is_timeout ( e ) :,if ignore_timeouts and is_noerr ( e ) :,"def read ( self , count = True , timeout = Non...","def read ( self , count = True , timeout = Non...",98.901377,98.91,False
1,1,"def _cache_mem ( curr_out , prev_mem , mem_len...",if prev_mem is None :,if mem_len < len ( curr_out ) :,"def _cache_mem ( curr_out , prev_mem , mem_len...","def _cache_mem ( curr_out , prev_mem , mem_len...",73.008389,94.86,False
2,2,def filtered ( gen ) : <TAB> for example in ge...,if example_len > max_length :,if example_len > max_length :,def filtered ( gen ) : <TAB> for example in ge...,def filtered ( gen ) : <TAB> for example in ge...,100.000000,100.00,True
3,3,"def search ( self , query ) : <TAB> # ""Search....","if item . get ( ""type"" , """" ) == ""audio"" :","if ""guide_id"" in item :","def search ( self , query ) : <TAB> # ""Search....","def search ( self , query ) : <TAB> # ""Search....",70.692051,91.97,False
4,4,"def _check_script ( self , script , directive ...","if var . must_contain ( ""/"" ) :",if var . name in self . _untrusted_vars :,"def _check_script ( self , script , directive ...","def _check_script ( self , script , directive ...",82.791727,96.05,False
...,...,...,...,...,...,...,...,...,...
4995,4995,def _super_function ( args ) : <TAB> passed_cl...,"if isinstance ( pyclass , pyobjects . Abstract...",if pyclass . is_subclass ( ) :,def _super_function ( args ) : <TAB> passed_cl...,def _super_function ( args ) : <TAB> passed_cl...,95.296605,95.33,False
4996,4996,def get_data ( row ) : <TAB> data = [ ] <TAB> ...,if result :,if result :,def get_data ( row ) : <TAB> data = [ ] <TAB> ...,def get_data ( row ) : <TAB> data = [ ] <TAB> ...,100.000000,100.00,True
4997,4997,"def say ( jarvis , s ) : <TAB> """"""Reads what i...",if not voice_state :,if voice_state :,"def say ( jarvis , s ) : <TAB> """"""Reads what i...","def say ( jarvis , s ) : <TAB> """"""Reads what i...",68.400323,97.95,False
4998,4998,"def __import__ ( name , globals = None , local...","if ""*"" in fromlist :","if ""*"" in fromlist :","def __import__ ( name , globals = None , local...","def __import__ ( name , globals = None , local...",100.000000,100.00,True


In [ ]:
df_results_final = df_results.drop(columns=["Unnamed: 0", "predicted_full_method", "target_full_method"])

In [ ]:
df_results_final.head()

,input_function,expected_if_statement,predicted_if_statement,code_bleu,bleu_4,exact_match
0,"def read ( self , count = True , timeout = Non...",if ignore_timeouts and is_timeout ( e ) :,if ignore_timeouts and is_noerr ( e ) :,98.901377,98.91,False
1,"def _cache_mem ( curr_out , prev_mem , mem_len...",if prev_mem is None :,if mem_len < len ( curr_out ) :,73.008389,94.86,False
2,def filtered ( gen ) : <TAB> for example in ge...,if example_len > max_length :,if example_len > max_length :,100.000000,100.00,True
3,"def search ( self , query ) : <TAB> # ""Search....","if item . get ( ""type"" , """" ) == ""audio"" :","if ""guide_id"" in item :",70.692051,91.97,False
4,"def _check_script ( self , script , directive ...","if var . must_contain ( ""/"" ) :",if var . name in self . _untrusted_vars :,82.791727,96.05,False


In [ ]:
df_results_final.to_csv("testset-results.csv")

In [ ]:
# overall codebleu over all the prediction-target pairs

with open("predictions.txt", "w", encoding="utf-8") as f:
    for pred in predicted_full_methods:
        f.write(pred + "\n")

with open("targets.txt", "w", encoding="utf-8") as f:
    for target in target_full_methods:
        f.write(target + "\n")

!cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && python calc_code_bleu.py --refs /content/targets.txt --hyp /content/predictions.txt --lang python --params 0.25,0.25,0.25,0.25

ngram match: 0.9574415034118224, weighted ngram match: 0.9564552493498323, syntax_match: 0.9316673403658838, dataflow_match: 0.7082488388827108
CodeBLEU score:  0.8884532330025623
